# Uso de DataBreachChecker para saber si tu cuenta ha sido hackeada

Vamos a utilizar la librería de Python que tiene OpenAI que nos facilita la vida para utilizar uno de los modelos de ChatGPT.

In [1]:
import requests
import os
import openai
import json
from openai import OpenAI

Necesitamos las api keys de OpenAi y de Rapid Api.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

Cargamos las variables de entorno por facilidad de uso. También usaremos el modelo "gpt-4o-mini" que es muy barato y para pruebas es posiblemente el mejor.

In [3]:
# Set your OpenAI and MyFitnessPal API keys
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
RAPIDAPI_KEY = os.environ.get('RAPIDAPI_KEY')

client = OpenAI()
model = "gpt-4o-mini"

En este caso, vamos a definir una única `tool` para el modelo y vamos a utilizar json. Lo que necesitamos es el email del usuario para pasárselo a la api.

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_email",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string"}
                },
            },
        }
    }
]

Ahora, toca escribir la función que contactará con la api de rapid api y DataBreachChecker, tenemos que pasarle el email como parámetro.

In [6]:
def check_email(email):
    url = "https://data-breach-checker.p.rapidapi.com/api/breach"
    headers = {
        "x-rapidapi-host": "data-breach-checker.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY,        
    }
    params = {
        "email": email
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API Error: {response.status_code} - {response.text}"}

Escribiré ahora una función para contactar con chatgpt.

In [10]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant that checks if the user email has been hacked. Use the supplied tools to check if the email was hacked."
    }
]

def chat(user_input):
    messages.append({
        "role": "user",
        "content": user_input
    })
        
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    
    return chat_completion

Ahora viene lo que es la interacción con el modelo. Preguntaremos al usuario por su email y el modelo devolverá la tool que debemos utilizar.

In [9]:
user_input = input("Please enter your email:").strip()

Please enter your email: miguel.lopez@deus.ai


In [11]:
chat_completion = chat(user_input)

In [13]:
print(chat_completion)

ChatCompletion(id='chatcmpl-AdCenPkLVp7ymo2tj2tuzpFNsRw5B', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NJK8hO431f9OCEoqoiSaTbbZ', function=Function(arguments='{"email":"miguel.lopez@deus.ai"}', name='check_email'), type='function')]))], created=1733906809, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_bba3c8e70b', usage=CompletionUsage(completion_tokens=21, prompt_tokens=71, total_tokens=92, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


Como podemos ver el modelo ha llamado a una `tool` la única que tenemos `check_email`. Es nuestro trabajo llamarla.

In [17]:
if not chat_completion.choices[0].message.tool_calls:
    print(chat_completion.choices[0].message.content)
else:
    function_call_result_messages = []
    for tool_call in chat_completion.choices[0].message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        print(f"Checking email: {arguments['email']}")
        email_data = check_email(arguments['email'])

        if not email_data['data']:
            content = "Email is not listed." 
        else: 
            f"Email is listed in {email_data['data'].count()} breaches."
        
        function_call_result_messages.append({
            "role": "assistant",
            "content": content
        })

print(function_call_result_messages)

Checking email: miguel.lopez@deus.ai
[{'role': 'assistant', 'content': 'Email is not listed.'}]


Ahora vamos a recrear el chat con los mensajes que envió el usuario pero el último mensaje añadirá la información del email y el modelo podrá responder al usuario.

In [18]:
completion_payload = {
    "model": model,
    "messages": messages + function_call_result_messages
}
print(completion_payload)

{'model': 'gpt-4o-mini', 'messages': [{'role': 'system', 'content': 'You are an assistant that checks if the user email has been hacked. Use the supplied tools to check if the email was hacked.'}, {'role': 'user', 'content': 'miguel.lopez@deus.ai'}, {'role': 'assistant', 'content': 'Email is not listed.'}]}


Volvemos a contactar con el modelo con estos datos.

In [19]:
response = client.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)

In [20]:
print(response.choices[0].message.content)

I checked the provided email address (miguel.lopez@deus.ai) for any known data breaches. There are no reported issues or hacks associated with this email. It appears to be secure for now. However, it's always a good practice to change your password regularly and enable two-factor authentication for added security.
